## Reflection Agent - Planner and Self-Critique Agents
### Web Research with Iterative Self-Improvement

Learning Objectives:
- Build a research agent with web search
- Implement critique agent for quality control
- Use reflection loop with max iterations

#### Real-World Use Cases:
1. **Content Research**: Gather and refine information
2. **Report Generation**: Iteratively improve quality
3. **Fact Checking**: Verify and enhance accuracy
4. **Competitive Analysis**: Research and critique findings

In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from typing_extensions import TypedDict, Annotated
import operator
from langgraph.graph import StateGraph, START, END

from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

# Configuration
BASE_URL = "http://localhost:11434"
MODEL_NAME = "qwen3"


llm = ChatOllama(model=MODEL_NAME, base_url=BASE_URL)

### Web Search Tool

In [ ]:
# DuckDuckGo search integration
from ddgs import DDGS

@tool
def web_search(query: str, num_results: int = 5) -> str:
    """Search the web using DuckDuckGo.
    
    Args:
        query: Search query string
        num_results: Number of results to return (default: 5)
    
    Returns:
        Formatted search results with titles, descriptions, and URLs
    """
    
    try:
        results = list(DDGS().text(query = query,
                                   max_results=num_results,
                                   region="us-en"))
        
        if not results:
            return f"No results found for '{query}'"
        
        formatted_results = [f"Search Results for '{query}':\n"]
        for i, result in enumerate(results, 1):
            title = result.get('title', 'No title')
            body = result.get('body', 'No description available')
            href = result.get('href', '')
            formatted_results.append(f"{i}. **{title}**\n   {body}\n   {href}")
        
        return "\n\n".join(formatted_results)
    
    except Exception as e:
        return f"Search error: {str(e)}"

In [ ]:
# test
web_search.invoke({'query': 'LangGraph tutorials', 'num_results': 3})

### Agent State

In [ ]:
# Create Agent State
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]
    research: str  # stores research output
    critique: str  # stores critique feedback
    iterations: int  # track iterations

### Researcher Node

In [ ]:
def researcher_node(state: AgentState):

    llm_with_tools = llm.bind_tools([web_search])

    critique = state.get('critique', '')
    iteration = state.get('iterations', 0)

    critique_context = ""
    if critique:
        critique_context = f"""
                Previous Critique: {critique}
                Address the missing points with new search queries.
                """

    system_prompt = SystemMessage(f"""
        You are a research agent with web search capabilities.
        {critique_context}
        INSTRUCTIONS:
        1. **MUST use web_search tool** first to gather information
        2. Provide comprehensive research based on search results

        Always call **web_search** before responding.
    """)

    messages = [system_prompt] + state['messages']

    response = llm_with_tools.invoke(messages)

    if hasattr(response, 'tool_calls') and response.tool_calls:
        for tc in response.tool_calls:
            print(f"[RESEARCHER] calling Tool {tc.get('name', '?')} with args {tc.get('args', '?')}")
    else:
        print(f"[RESEARCHER] Iteration {iteration + 1} - Research complete")

    return {'messages': [response]}

### Critique Node

In [ ]:
def critique_node(state: AgentState):
    
    messages = state['messages']
    iteration = state.get('iterations', 0)
    
    # extract research from messages
    research_content = ""
    for msg in reversed(messages):
        if hasattr(msg, 'content') and msg.content:
            research_content = msg.content
            break
    
    system_prompt = SystemMessage("""
        You are a critique agent. Evaluate if research is good enough.
        
        Check:
        1. Does it answer the main question?
        2. Is there reasonable detail?
        
        Response Format:
        DECISION: APPROVE or REVISE
        
        Be lenient. APPROVE if research is decent enough.
        Only REVISE if critical information is completely missing.
    """)
    
    critique_prompt = HumanMessage(f"""
        Evaluate this research:
        
        {research_content}
    """)
    
    response = llm.invoke([system_prompt, critique_prompt])
    
    print(f"[CRITIQUE] Iteration {iteration + 1}")
    
    return {
        'critique': response.content,
        'research': research_content,
        'iterations': iteration + 1
    }

### Routing Logic

In [ ]:
# Routing from researcher
def should_continue(state: AgentState):
    last = state['messages'][-1]
    
    if hasattr(last, 'tool_calls') and last.tool_calls:
        return "tools"
    else:
        return "critique"

In [ ]:
# Routing from critique
MAX_ITERATIONS = 10

def check_approval(state: AgentState):
    
    critique = state.get('critique', '')
    iterations = state.get('iterations', 0)
    
    # max iterations reached
    if iterations >= MAX_ITERATIONS:
        print(f"[SYSTEM] Max iterations ({MAX_ITERATIONS}) reached. Stopping.")
        return END
    
    # check if approved
    if 'APPROVE' in critique.upper():
        print(f"[SYSTEM] Research approved after {iterations} iteration(s)")
        return END
    else:
        print(f"[SYSTEM] Revision needed. Continuing iteration {iterations + 1}")
        return "researcher"

### Build Graph

In [ ]:
# =============================================================================
# Graph
# =============================================================================
def create_agent():

    builder = StateGraph(AgentState)

    builder.add_node("researcher", researcher_node)
    builder.add_node("tools", ToolNode([web_search]))
    builder.add_node("critique", critique_node)

    builder.add_edge(START, "researcher")
    builder.add_conditional_edges("researcher", should_continue, ["tools", "critique"])
    builder.add_edge("tools", "researcher")
    builder.add_conditional_edges("critique", check_approval, ["researcher", END])

    graph = builder.compile()

    return graph

In [ ]:
agent = create_agent()
agent

### Run Agent

In [ ]:
query = "What are the latest developments in LangGraph for building AI agents?"

result = agent.invoke({
    'messages': [HumanMessage(query)],
    'research': '',
    'critique': '',
    'iterations': 0
})

In [ ]:
# Final research output
print("\n" + "="*80)
print("FINAL RESEARCH OUTPUT")
print("="*80 + "\n")
print(result['research'])

In [ ]:
# result

In [ ]:
query = "Do research on Nvidia stock performance and recent news."

result = agent.invoke({
    'messages': [HumanMessage(query)],
    'research': '',
    'critique': '',
    'iterations': 0
})

In [ ]:
print(result['research'])

In [ ]:
print(result['critique'])